In [ ]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation

from tensorflow.keras.optimizers import RMSprop
from keras.optimizers import RMSprop





In [ ]:
file_path = tf.keras.utils.get_file('shakesphere', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
text = open(file_path, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]
characters = sorted(set(text))


In [ ]:
char_to_index= dict((c,i) for i,c in enumerate(characters))
index_to_char= dict((i,c) for i,c in enumerate(characters))

SEQ_LENGTH = 40
STEP_SIZE = 3


sentences = []
next_characters=[]

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i:i+SEQ_LENGTH])
  next_characters.append(text[i:i+SEQ_LENGTH])



In [ ]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)


In [ ]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_chars = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_chars.append(text[i + SEQ_LENGTH])

print(f"Total sequences: {len(sentences)}")


x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1


Total sequences: 166654


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          86,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 39)                  │           5,031 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 39)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 91,047 (355.65 KB)

 Trainable params: 91,047 (355.65 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x, y, batch_size=256, epochs=10)


Epoch 1/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 113s 170ms/step - loss: 2.4765
Epoch 2/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 145s 175ms/step - loss: 1.7641
Epoch 3/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 140s 172ms/step - loss: 1.6106
Epoch 4/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 142s 172ms/step - loss: 1.5284
Epoch 5/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 142s 172ms/step - loss: 1.4707
Epoch 6/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 141s 171ms/step - loss: 1.4294
Epoch 7/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 143s 173ms/step - loss: 1.4023
Epoch 8/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 142s 173ms/step - loss: 1.3800
Epoch 9/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 111s 171ms/step - loss: 1.3603
Epoch 10/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 143s 173ms/step - loss: 1.3418


In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature=0.5):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence

    for i in range(length):
        x_pred = np.zeros((1, SEQ_LENGTH, len(characters)))

        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_pred, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_char = index_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

    return generated


In [ ]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.6))
print(generate_text(300, 0.8))


 a daughter;
here 'tis; commends it to your channer,
and the fight and the company of the country
on the conceester'd thee that the earth,
the company from the company of the country
that the country's seas the fiert than you have not.

duke of york:
and i will not i stay the conceed and the conceite.

romeo:
and i will be so to the broth
lood thou ne'er put'st back.

king edward iv:
i do the sun them that every to the seass
be not unto the earth of my father of the earth,
the grace and lead not the nease, the sunss
the boys and the friends up the earth,
and in the earth of the great ground for not.

warwick:
my lord, my lord, and let the bear thee makes:
the great to the 
how god and good men hate so foul a liar,
than that the matters the father was wrong,
shall not so now i have be prayens and dismore,
cleft thou must death.

duke of york:
the can beheat night it, the whither do your forther.

capulet:
for the friends it defend and not a more.

romeo:
if the rest to the fresh them, t

In [ ]:
model.save('shakespeare_lstm.h5')
model = tf.keras.models.load_model('shakespeare_lstm.h5')


In [ ]:
model.save("my_model.keras")  # Recommended


In [ ]:
from tensorflow import keras

model = keras.models.load_model("my_model.keras")  # Load the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
